# MQ-RNN model

In [1]:
#!pip install deepts_forecasting

### Import libraries

In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from deepts_forecasting.utils.data import TimeSeriesDataSet
from deepts_forecasting.utils.data.encoders import TorchNormalizer
from deepts_forecasting.datasets import AirPassengersDataset
from deepts_forecasting.models.mqrnn import MQRNN


### Dataset

In [3]:
data = AirPassengersDataset().load()
data['year'] = data['Month'].dt.year
data['month'] = data['Month'].dt.month
data['group'] = '0'
data['time_idx'] = np.arange(len(data))
data['Passengers'] = data['Passengers'].astype(float)
data['month'] = data['month'].astype('str')
data.head()

,Month,Passengers,year,month,group,time_idx
0,1949-01-01,112.0,1949,1,0,0
1,1949-02-01,118.0,1949,2,0,1
2,1949-03-01,132.0,1949,3,0,2
3,1949-04-01,129.0,1949,4,0,3
4,1949-05-01,121.0,1949,5,0,4


### Split train/test sets

In [4]:
max_encoder_length = 18
max_prediction_length = 12

training_cutoff = data["time_idx"].max() - max_encoder_length - max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    max_encoder_length= max_encoder_length,
    min_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    min_prediction_length=max_prediction_length,
    time_idx="time_idx",
    target="Passengers",
    group_ids=["group"],
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=['month'],
    time_varying_known_reals=[],
    time_varying_unknown_reals=["Passengers"],
    time_varying_unknown_categoricals=[],
    target_normalizer=TorchNormalizer(method="standard",
                                      transformation=None),
    )

training.get_parameters()
validation = TimeSeriesDataSet.from_dataset(training,
                                            data[lambda x: x.time_idx > training_cutoff])

batch_size = 16
train_dataloader = DataLoader(training, batch_size=batch_size, shuffle=False, drop_last=False)
val_dataloader = DataLoader(validation, batch_size=batch_size, shuffle=False, drop_last=False)


### Define model

In [26]:
pl.seed_everything(1234)
# create PyTorch Lighning Trainer with early stopping
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4,
                                    patience=60, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

trainer = pl.Trainer(
    max_epochs=300,
    gpus=0,  # run on CPU, if on multiple GPUs, use accelerator="ddp"
    gradient_clip_val=0.1,
    limit_train_batches=30,  # 30 batches per epoch
    callbacks=[lr_logger, early_stop_callback],
    logger=TensorBoardLogger("lightning_logs")
)

model = MQRNN.from_dataset( 
    training,
    quantiles=[0.25, 0.5, 0.75],
    hidden_size=16, 
    rnn_layers=2, 
)
model.summarize

Global seed set to 1234
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


<bound method LightningModule.summarize of MQRNN(
  (loss): QuantileLoss()
  (logging_metrics): ModuleList()
  (global_mlp): Linear(in_features=22, out_features=208, bias=True)
  (local_mlp): Linear(in_features=390, out_features=3, bias=True)
  (encode_rnn): LSTM(7, 16, num_layers=2, batch_first=True)
  (embeddings): ModuleDict(
    (month): Embedding(12, 6)
  )
)>

In [27]:
model.hparams

"categorical_groups":                {}
"embedding_labels":                  {'month': array(['1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'],
      dtype=object)}
"embedding_paddings":                []
"embedding_sizes":                   {'month': [12, 6]}
"hidden_size":                       16
"learning_rate":                     0.001
"log_interval":                      -1
"log_val_interval":                  None
"logging_metrics":                   ModuleList()
"loss":                              QuantileLoss()
"max_encoder_length":                18
"max_prediction_length":             12
"monotone_constaints":               {}
"output_size":                       1
"output_transformer":                TorchNormalizer()
"quantiles":                         [0.25, 0.5, 0.75]
"rnn_layers":                        2
"static_categoricals":               []
"static_reals":                      []
"time_varying_categoricals_decoder": ['month']
"time_varying_categoric

### Train model with early stopping

In [28]:
trainer.fit(
    model, train_dataloader=train_dataloader, val_dataloaders=val_dataloader,
)

# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = MQRNN.load_from_checkpoint(best_model_path)

# calcualte mean absolute error on validation set
actuals = torch.cat([model.transform_output(prediction=y, target_scale=x['target_scale'])
                     for x, y in iter(val_dataloader)])
predictions,x_index = best_model.predict(val_dataloader)
mae = (actuals - predictions[:, :, 1:2]).abs().mean()
# print('predictions shape is:', predictions.shape)
# print('actuals shape is:', actuals.shape)
print(torch.cat([actuals, predictions[:, :, 1:2]]))
print('MAE is:', mae)


  | Name            | Type         | Params
-------------------------------------------------
0 | loss            | QuantileLoss | 0     
1 | logging_metrics | ModuleList   | 0     
2 | global_mlp      | Linear       | 4.8 K 
3 | local_mlp       | Linear       | 1.2 K 
4 | encode_rnn      | LSTM         | 3.8 K 
5 | embeddings      | ModuleDict   | 72    
-------------------------------------------------
9.8 K     Trainable params
0         Non-trainable params
9.8 K     Total params
0.039     Total estimated model params size (MB)


Global seed set to 1234


Epoch 0:  86%|██████████▎ | 6/7 [00:00<00:00, 46.50it/s, loss=1.25, v_num=16]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|█| 7/7 [00:00<00:00, 49.64it/s, loss=1.25, v_num=16, val_loss=4
Epoch 1:  86%|▊| 6/7 [00:00<00:00, 40.53it/s, loss=1.19, v_num=16, val_loss=4
Validating: 0it [00:00, ?it/s]
Epoch 1: 100%|█| 7/7 [00:00<00:00, 43.61it/s, loss=1.19, v_num=16, val_loss=4
Epoch 2:  86%|▊| 6/7 [00:00<00:00, 37.15it/s, loss=1.15, v_num=16, val_loss=4
Validating: 0it [00:00, ?it/s]
Epoch 2: 100%|█| 7/7 [00:00<00:00, 39.88it/s, loss=1.15, v_num=16, val_loss=4
Epoch 3:  86%|▊| 6/7 [00:00<00:00, 35.29it/s, loss=1.19, v_num=16, val_loss=4
Validating: 0it [00:00, ?it/s]
Epoch 3: 100%|█| 7/7 [00:00<00:00, 37.83it/s, loss=1.19, v_num=16, val_loss=4
Epoch 4:  86%|▊| 6/7 [00:00<00:00, 28.77it/s, loss=1.13, v_num=16, val_loss=4
Validating: 0it [00:00, ?it/s]
Epoch 4: 100%|█| 7/7 [00:00<00:00, 31.24it/s, loss=1.13, v_num=16, val_loss=3
Epoch 5:  86%|▊| 6/7 [00:00<00:00, 36.03it/s, loss=1.1, v_num=16,